In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：
bounds = [(-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10)]
x_opt = [2.33049935147405174, 1.95137236847114592, -0.477541399510615805, 4.36572624923625874, -0.624486959100388983,
         1.03813099410962173, 1.5942266780671519]
y_opt = 680.630057374402


def G09(X):
    y = (X[0] - 10) ** 2 + 5 * (X[1] - 12) ** 2 + X[2] ** 4 + 3 * (X[3] - 11) ** 2 + 10 * (X[4] ** 6) + 7 * (
                X[5] ** 2) + X[6] ** 4 - 4 * X[5] * X[6] - 10 * X[5] - 8 * X[6]

    g1 = -127 + 2 * X[0] ** 2 + 3 * X[1] ** 4 + X[2] + 4 * X[3] ** 2 + 5 * X[4]
    g2 = -282 + 7 * X[0] + 3 * X[1] + 10 * X[2] ** 3 + X[3] - X[4]
    g3 = -196 + 23 * X[0] + X[1] ** 2 + 6 * X[5] ** 2 - 8 * X[6]
    g4 = 4 * X[0] ** 2 + X[1] ** 2 - 3 * X[0] * X[1] + 2 * X[2] ** 2 + 5 * X[5] - 11 * X[6]

    return y, g1, g2, g3, g4

In [4]:
# y = lambda X: G09(X)[0]
# g1 = lambda X: G09(X)[1]
# g2 = lambda X: G09(X)[2]
# g3 = lambda X: G09(X)[3]
# g4 = lambda X: G09(X)[4]
# x0 = [-10,-10,-10,-10,-10,-10,-10]
# x0 = [0,0,0,0,0,0,0]
# x0 = [10,10,10,10,10,10,10]
# # x0 = [1,2,3,4,5,6,7]

In [5]:
# maxfunction(g4,x0,bounds)

In [6]:
### 适应度函数
def fitnessG09_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG09_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 10024263
    fmin = 7
    vmax = 14845
    e = 0.1
    n = 4
    
    res = G09(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:5]>0)
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness  


In [7]:
X = [bounds[i][0] for i in range(len(bounds))]
print(fitnessG09_vio(X))

-260985535.2532839


In [8]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG09_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.85
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG09_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [9]:
PSO = PSO_fit(pN=15000, dim=7, max_iter=500000, bounds=bounds)

In [10]:
PSO.init_Population()

In [11]:
PSO.iterator()

[-1.28869463e+04 -4.15938910e+07 -4.13843865e+07 ... -4.07998923e+07
 -1.13778114e+04 -4.39117308e+07]
[-1.28869463e+04 -1.00013109e+07 -4.13843865e+07 ... -4.07998923e+07
 -1.13778114e+04 -4.07659692e+07]
[-1.28869463e+04 -1.00013109e+07 -4.02984492e+07 ... -1.26768414e+04
 -1.13778114e+04 -4.07067196e+07]
[-1.28869463e+04 -1.00013109e+07 -4.02984492e+07 ... -1.26768414e+04
 -1.13778114e+04 -3.67684156e+04]
[-1.28869463e+04 -1.00013109e+07 -1.64835823e+04 ... -1.26768414e+04
 -1.13778114e+04 -3.81444695e+03]
[ -4148.82590518  -5477.28466532 -14239.97235306 ... -12676.84138302
 -11377.81141593  -3814.44695084]
[ -1369.96197502  -1928.96864787 -11239.3517127  ... -12676.84138302
 -11377.81141593  -3814.44695084]
[ -1369.96197502  -1928.96864787 -11239.3517127  ...  -1648.74914344
 -11258.17971299  -3814.44695084]
[-1369.96197502 -1928.96864787 -1268.15677432 ... -1648.74914344
 -3775.01087619 -3814.44695084]
[-1369.96197502 -1928.96864787 -1268.15677432 ... -1648.74914344
 -2588.7543778

[-687.25664189 -687.61638256 -693.56117099 ... -687.90875814 -688.05968528
 -690.80209369]
[-687.25664189 -687.61638256 -693.56117099 ... -687.90875814 -688.05968528
 -690.80209369]
[-687.25664189 -687.61638256 -693.56117099 ... -687.90875814 -688.05968528
 -690.80209369]
[-687.25664189 -687.61638256 -693.56117099 ... -687.90875814 -688.05968528
 -690.80209369]
[-687.25664189 -687.61638256 -693.56117099 ... -687.90875814 -688.05968528
 -690.80209369]
[-687.25664189 -687.61638256 -690.57236094 ... -687.90875814 -687.44626679
 -690.80209369]
[-687.25664189 -687.61638256 -690.57236094 ... -687.90875814 -687.44626679
 -690.80209369]
[-687.25664189 -687.61638256 -688.34891991 ... -684.81075868 -687.44626679
 -689.49595167]
[-687.25664189 -687.61638256 -687.38485414 ... -684.81075868 -686.01511619
 -689.49595167]
[-687.25664189 -687.61638256 -687.38485414 ... -684.81075868 -686.01511619
 -689.49595167]
[-687.25664189 -687.61638256 -687.38485414 ... -684.81075868 -686.01511619
 -687.80764157]

[-681.41112583 -681.16494543 -681.27845175 ... -681.77186516 -681.67646362
 -681.24023802]
[-681.41112583 -681.16494543 -681.27845175 ... -681.77186516 -681.67646362
 -681.24023802]
[-681.41112583 -681.16494543 -681.27845175 ... -681.77186516 -681.67646362
 -681.24023802]
[-681.41112583 -681.16494543 -681.27845175 ... -681.77186516 -681.67646362
 -681.24023802]
[-681.41112583 -681.16494543 -681.27845175 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.27845175 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.27845175 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.24023802 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.24023802 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.24023802 ... -681.77186516 -681.03155436
 -681.24023802]
[-681.3593922  -681.16494543 -681.24023802 ... -681.77186516 -681.03155436
 -681.24023802]

[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.79023294]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.75364254]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.73119639]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.73119639]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.73119639]
[-680.77931044 -680.72737302 -680.72600458 ... -680.78660183 -680.72692512
 -680.73119639]

[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.68531015]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.66310289]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.66310289]
[-680.65597273 -680.66118803 -680.66124011 ... -680.6613712  -680.66179885
 -680.66310289]

...

[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]

[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]

[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]

[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]
[-680.63005737 -680.63005737 -680.63005737 ... -680.63005737 -680.63005737
 -680.63005737]


KeyboardInterrupt: 